![](jpg/stock_small.jpg)

This is the second part of a multi-part series on Snowpark. In the first part I have reviewed a [Quick Start](#Quick-Start) guide, i.e. how to set up get started. The Quick Start steps will be exactly the same for this notebook. 

In this session, the focus will be

- [Quick Start](#Quick-Start)
- [Advanced API features and Visualization](#Advanced-API-features-and-Visualization)
- [User Defined Functions](#User-Defined-Functions)

All notebooks in this series require a Jupyter notebook environment with a Scala kernel. If you do not already have access to that type of environment I would higly recommend to use [Snowtire V2](https://github.com/zoharsan/snowtire_v2) and this excellent [post](https://medium.com/snowflake/from-zero-to-snowpark-in-5-minutes-72c5f8ec0b55). Additional instructions on versions uses in this series as well as how to make the notebook look nicer by using nbextensions can be found in this github repo.

Versions used in this notebook are up-to-date as of August 2021. Please update them as necessary in the Snowtire setup step.

In case you see any unexpected errors, restart the kernel via *Kernel -> Restart* and start the notebook from the beginning.

Lastly, in my experience, when running on the Almond Kernel most problems reported by Ammonite REPL can be resolved by deleting the Ammonite cache and restarting the Kernel (as mentioned above).

        docker exec -d SnowTrekPost rm -rf /home/jovyan/.cache/almond


# Quick Start

First, we have to set up the Environment for our Notebook. The instructions for setting up the environment are [here](https://docs.snowflake.com/en/developer-guide/snowpark/quickstart-jupyter.html#configuring-the-jupyter-notebook-for-snowpark)

## Step 1

Configure the notebook to use a Maven repository for a library that Snowpark depends on.

In [ ]:
import sys.process._
val osgeoRepo = coursierapi.MavenRepository.of("https://repo.osgeo.org/repository/release")
interp.repositories() ++= Seq(osgeoRepo)

## Step 2

Create a directory (if it doesn't exist) for temporary files created by the [REPL](https://ammonite.io/#Ammonite-REPL) environment. To avoid any side-effects from previous runs, we also delete any files that might exist in that directory.

**Note: Make sure that you have the operating system permissions to create a directory in that location.**

**Note: If you are using multiple notebooks, you’ll need to create and configure a separate REPL class directory for each notebook.**

In [ ]:
import ammonite.ops._
import ammonite.ops.ImplicitWd._

// This folder is used to store generated repl classes, which will later be used in UDF.
// Please provide an empty folder path.This is essential for Snowpark UDF to work
val replClassPath = pwd+"/repl_classes"

// delete any old files in the directory
import sys.process._
s"rm -rf $replClassPath" !

// Create the repl class folder
import sys.process._
s"mkdir -p $replClassPath" !

## Step 3

Configure the compiler for the Scala REPL. This does the following:
- Configures the compiler to generate classes for the REPL in the directory that you created earlier.
- Configures the compiler to wrap code entered in the REPL in classes, rather than in objects.
- Adds the directory that you created earlier as a dependency of the REPL interpreter.

In [ ]:
// Generate all repl classes in repl class folder
interp.configureCompiler(_.settings.outputDirs.setSingleOutput(replClassPath))
interp.configureCompiler(_.settings.Yreplclassbased.value = true)
interp.load.cp(os.Path(replClassPath))

## Step 4
Import the Snowpark library from Maven.

In [ ]:
import $ivy.`com.snowflake:snowpark:0.8.0`

Let's recreate 2 directories for later use, one for data files and one for user credentials.

In [ ]:
import ammonite.ops._
import ammonite.ops.ImplicitWd._

%%('bash, "-c", "[ -d data ] && echo 'data exists' || ln -s ~/snowtrek/data data")
%%('bash, "-c", "[ -d creds ] && echo 'creds exists' || ln -s ~/snowtrek/creds creds")

To create a session we need to authenticate ourselves to the Snowflake instance. Though it might be tempting to just override the authentication variables below with hard coded values, its not considered best practice to do so. In case you ever wanted to share your version of the notebook, your could disclose your credentials by mistake to the recipient. Even worse, if you upload your notebook to a public code repository, you might advertise your credentials to the whole wide world. To prevent that, you should keep your credentials in an external file (like we are doing here).

Then update your credentials in that file and they will be save on your local machine. Even better if you do not use user/password authentication but [private key authentication](https://docs.snowflake.com/en/user-guide/key-pair-auth.html). 

Copy the credentials template file creds/template_credentials.txt to creds/credentials.txt and update the file with your credentials. Put your key files into the same directory or update the location accordingly in your credentials file. 

In [ ]:
import com.snowflake.snowpark._
import com.snowflake.snowpark.functions._

val session = Session.builder.configFile("creds/credentials.txt").create

## Step 5
Add the Ammonite kernel classes as dependencies for your UDF

In [ ]:
def addClass(session: Session, className: String): String = {
  var cls1 = Class.forName(className)
  val resourceName = "/" + cls1.getName().replace(".", "/") + ".class"
  val url = cls1.getResource(resourceName)
  val path = url.getPath().split(":").last.split("!").head
  session.addDependency(path)
  path
}
addClass(session, "ammonite.repl.ReplBridge$")
addClass(session, "ammonite.interp.api.APIHolder")
addClass(session, "pprint.TPrintColors")

## Step 6

For this exercise we need three additional libraries
- [plotl-scala (Plotly for Scala)](https://github.com/alexarchambault/plotly-scala)
- [Spire (Numeric Abstractions for Scala)](https://typelevel.org/spire/)
- [Stanford CoreNLP](https://stanfordnlp.github.io/CoreNLP/)


In [ ]:
import $ivy.`org.plotly-scala::plotly-almond:0.8.2`
import $ivy.`org.typelevel::spire:0.17.0`
import $ivy.`edu.stanford.nlp:stanford-corenlp:4.2.2`

import coursierapi._
interp.load.ivy(
      Dependency.of("edu.stanford.nlp", "stanford-corenlp", "4.2.2").withClassifier("models"),
) 

# Advanced API features and Visualization

We completed the first part of this series using the Snowpark dataframe interface to project and filter datasets via the Snowpark databframe API. In the following section we will learn about more advanced features of the dataframe API like aggregations and pivot. Secondly, instead of just producing tabular result sets, we will visualize the results using a graphics package called [plotly](https://github.com/alexarchambault/plotly-scala). 

The for this section is to produce a dataframe that shows the count of open orders, filled orders by order date. The dataaframe should look similar to the matrix below

    -------------------------------------------------
    |"O_ORDERDATE"  |"OPEN_COUNT"  |"FILLED_COUNT"  |
    -------------------------------------------------
    |1996-06-23     |6242          |0               |
    |1995-12-05     |6306          |0               |
    |1995-07-18     |6236          |0               |
    |1994-09-06     |0             |6285            |
    |1992-04-22     |0             |6341            |
    -------------------------------------------------

We then want to visualize those counts via a line chart. 

Let's see how we can do that in Snowpark.

In [ ]:
val size:String="10"
val demoDataSchema:Seq[String]=Seq("SNOWFLAKE_SAMPLE_DATA","TPCH_SF"+size)
val demoOrdersDf=session.table(demoDataSchema :+ "ORDERS")

In the next cell, we will filter the Orders by Status and only return Orders having the folling status values. 

    open ("O") 
    filled ("F") 
    
We then select only the 3 columns we are interested in

    O_ORDERDATE
    O_ORDERSTATUS
    O_ORDERKEY
    
Then we count the resulting rows by 

    date 
    status
    
This returns the count of Orders by Date and Status. 

In [ ]:
val demoOrdersGroupedDf=
        demoOrdersDf
            .filter((col("O_ORDERSTATUS")===lit("O") || col("O_ORDERSTATUS") === "F"))
            .select(col("O_ORDERDATE"),col("O_ORDERSTATUS"),col("O_ORDERKEY"))
            .groupBy(col("O_ORDERDATE"),col("O_ORDERSTATUS"))
            .agg(count_distinct(col("O_ORDERKEY")).name("O_COUNT"))

demoOrdersGroupedDf.schema

However, to visualize the dataset it would be more convenient to have one row per day with the count of open and filled orders. We accomplish that by using the pivot function. 


In [ ]:
val demoOrdersGroupedPivotDf=
        demoOrdersGrouped
            .pivot(col("O_ORDERSTATUS"),Seq('F','O'))
            .sum(col("O_COUNT"))
            .select(col("O_ORDERDATE"),coalesce(col("'O'"),lit(0)).name("OPEN_COUNT"),coalesce(col("'F'"),lit(0)).name("FILLED_COUNT"))

demoOrdersGroupedPivotDf.schema

Viola! You hae created the data exactly as we designed it at the beginning. The last step is to visualize the data. 

In general, when we want to display a line chart for a single metric, we need 2 vectors of data. 
- The first vector is a Sequence of strings, also called Labels. 
- The second vector is a sequence of values, which have to be of data type double.
If we have two metrics, we repeat the above structure of labels and values and wrap the two metrics in a sequence.

So for our example we have the following structure:
- Metric1: Open Orders
  - Labels: Days
  - Values: Open_Count
- Metric2:
  - Labels: Days
  - Values: Filled_Count
  
As you can see, displaying the data is a breeze since we have prepared our data well. 
  

In [ ]:
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

{
    val data=demoOrdersGroupedPivot
        .sort(col("O_ORDERDATE"))
        .collect()

    val days=data.map(r => r(0).toString).toSeq
    val open_count=data.map(r => r(1).toString.toDouble).toSeq
    val filled_count=data.map(r => r(2).toString.toDouble).toSeq

    plot(
        Seq(
          Scatter(days,open_count,name="OPEN_COUNT")
         ,Scatter(days,filled_count,name="FILLED_COUNT"))
         ,Layout(
             title="Orders By Date",
             xaxis=Axis(title="Date"),
             yaxis=Axis(title="Orders per day"),
        )
    )
}

For a more in depth discussions on the features of plotly, please review the [documentation](https://github.com/alexarchambault/plotly-scala). 

# User Defined Funtions

## Processing of Rational Numbers with Arbitrary Precision

In the next section we will introduce another powerfull feature in Snowpark, i.e user defined functions also called UDFs. UDFs allow us to express arbitrary logic via Scala and execute that logic against massive datasets. The beauty is that the data doesn't have to move to the client machine but the necessary jar files will be deployed to Snowflake automatically by Snowpark. We just have to let Snowpark know how to resolve the dependencies. And we will learn how to do that in this section. 

Lets have a look!

In this example we will learn how to process Rational numbers with arbitrary precission. As you may know, Snowflake uses, besides other data types, double-precision (64 bit) IEEE 754 floating-point numbers. More about this topic can be found [here](https://docs.snowflake.com/en/sql-reference/data-types-numeric.html#float-float4-float8).

So lets look at the following example:

        12345678909876543219999-12345678909876543210000
        
In the following statement we have 2 numbers in string representation. We will cast those numbers to a floating point representation and subtract them from each other. 

In [ ]:
val longNumbers=session.createDataFrame(Seq(("12345678909876543.9999","12345678909876543.0000"))).toDF("S1","S2")

longNumbers
    .withColumns(Seq("FP1","FP2"),Seq(callBuiltin("TO_DOUBLE",col("S1")),callBuiltin("TO_DOUBLE",col("S2"))))
    .withColumn("RESULT",col("FP1")-col("FP2"))
    .select(col("FP1"),col("FP2"),col("RESULT"))
    .show()


So to avoid that problem we have to use numbers with an unbound precision. The OSS package [Spire](https://typelevel.org/spire/guide.html) addresses exactly that problem.

In the following example I'll show how easy it is to take advantage of Spire to enjoy lossless processing of numbers with unbound precision. Note that we have already imported the package in [Step 6 of the Quick Start](#Step-6) .

Creating UDF for Snowpark is straight forward. You can review the documentation [here](https://docs.snowflake.com/en/developer-guide/snowpark/creating-udfs.html#creating-udfs-in-jupyter-notebooks). Please note that the object / class needs to extend *Serializable*. For this example I am creating and object instead of a class since we need only one instance of the snowmath class (check [here](https://docs.scala-lang.org/tour/singleton-objects.html) for more details).

The implemention of our object is very straight forward. We will use spire data type *Rational* and map the add/subtract/divide/multiply primitives to the corresponding spire functions. As a convenience function I have added a cast from Rational to Decimal just in case we wanted to see the value of a Rational number with a specific precision.

In [ ]:
object snowmath extends Serializable {
    
    import spire.algebra._
    import spire.math._
    import spire.implicits._
    import java.math.{MathContext, RoundingMode}  
    
    def opRational (op:String,r1:String,r2:String):String = {
        op match {
            case "add"      => (Rational(r1)+Rational(r2)).toString()
            case "subtract" => (Rational(r1)-Rational(r2)).toString()
            case "divide"   => (Rational(r1)/Rational(r2)).toString()
            case "multiply" => (Rational(r1)*Rational(r2)).toString()
            case _ => "op not found"
        }
    }
    def fromRationalToBigDecimal(precision:String,r:String):String = {
        Rational(r).toBigDecimal(precision.toInt,RoundingMode.HALF_EVEN).toString()
    }

}

Rational Number in Spire are represented as n/d. This means, we have to change the first number to a corresponding expression

    12345678909876543.9999 => 123456789098765439999/10000
    
And Voila, we get the expected result since spire can handle Rational numbers with unbound precision.

In [ ]:
snowmath.opRational("subtract","123456789098765439999/10000","12345678909876543")


However, the cell above was running on our local machine. To execute the object in Snowflake we have to define the dependencies for our object above, and we have to create UDF mappings for the functions we want to call in Snowflake. Snowpark will then upload all necessary jar files to Snowflake and create the necessary mapping function for calling the scala object directly in SQL. This step could take some time depending on your ISP's upload speed.

In [ ]:
addClass(session,"spire.math.Rational")
addClass(session,"algebra.ring.Field")
addClass(session,"cats.kernel.Order")

val opRationalUdf=session.udf.registerTemporary((op:String,fp1:String,fp2:String) => snowmath.opRational(op,fp1,fp2))
val fromRationalToBigDecimalUdf=session.udf.registerTemporary((p:String,r:String) => snowmath.fromRationalToBigDecimal(p,r))


To execute the opRationalUdf in a sql statement, 

- we create a Snowpark dataframe with our input numbers, 
- we create a new column which is the result from the computation
- we create a another column casting the result to a decimal with 40 digit precision


In [ ]:
val longNumbersDf=session.createDataFrame(Seq(("123456789098765439999/10000","12345678909876543"))).toDF("R1","R2")

longNumbersDf
    .withColumn("RESULT",opRationalUdf(lit("subtract"),col("R1"),col("R2")))
    .withColumn("RESULT_DECIMAL",fromRationalToBigDecimalUdf(lit("40"),col("RESULT")))
    .show()

As you can see above, snowmath can handle Rational numbers with unbound precision. The decimal result is still a string, however it could be cast to a floating point number in Snowflake. Just remember, that Snowflake floating point numbers have a maximum number of significant digits.

## Sentiment Analysis

This example is similar to the previous though it shows a couple more features. 

- more complex UDF code
- building a complex return object
- lazy loading of data files

The goal is to compute the sentiment of a text segment using the [Stanford CoreNLP library]((https://stanfordnlp.github.io/CoreNLP/) and return a JSON object that lists the sentence and the sentiment as follows:

    [
        {
            "sentence": "Happy Days",
            "sentiment": "Positive"
        }
    ]

Note that we have already imported the package in Step 6 of the Quick Start .

Note the definition of *pipeline* and the fact, that *pipeline* is a class variable, i.e. its defined outside of the UDF *compute*. The initilization code for *pipeline* will be executed upon the first invocation of the Sentiment object. This is important to understand, since Snowflake allows a certain amount of time for class initialization (currently 300 secs) but only 5 secs for the method execution. So if *pipeline* was initialized within the scope of the *compute* method, it most likely would exceed the 5 second timeout and fail when executed in Snowflake. 

In [ ]:
object Sentiment extends Serializable {

    import java.io.File
    import java.nio.charset.Charset
    import java.util.Properties

    import edu.stanford.nlp.coref.CorefCoreAnnotations
    import edu.stanford.nlp.ling.CoreAnnotations
    import edu.stanford.nlp.neural.rnn.RNNCoreAnnotations
    import edu.stanford.nlp.pipeline.{Annotation, StanfordCoreNLP}
    import edu.stanford.nlp.sentiment.SentimentCoreAnnotations
    import edu.stanford.nlp.sentiment.SentimentCoreAnnotations.SentimentAnnotatedTree
    import edu.stanford.nlp.util.CoreMap

    import scala.collection.JavaConverters._
    
    import com.snowflake.snowpark.types._

    private def getSentiment(sentiment: Int): String = sentiment match {
        case x if x == 0 || x == 1 => "Negative"
        case 2 => "Neutral"
        case x if x == 3 || x == 4 => "Positive"
    }

    private val props: Properties = new Properties()
    props.put("annotators", "tokenize, ssplit, parse, sentiment")
 
    private lazy val pipeline: StanfordCoreNLP = new StanfordCoreNLP(props)
    
    def compute (text:String):String = {

        // create blank annotator
        val document: Annotation = new Annotation(text)
        
        // run all Annotators
        pipeline.annotate(document)

        val sentences: List[CoreMap] = document.get(classOf[CoreAnnotations.SentencesAnnotation]).asScala.toList

        "[\n" + 
            sentences
              .map(sentence => (sentence, sentence.get(classOf[SentimentAnnotatedTree])))
              .map { case (sentence, tree) => "{\n"+
                                                "  \"sentence\": \""+sentence.toString + "\",\n" + 
                                                "  \"sentiment\": \""+getSentiment(RNNCoreAnnotations.getPredictedClass(tree))+ "\"\n" +
                                               "}"
                   }
              .mkString(",") +
        "\n]"
        
    }
}

Let's test our Scala object locally with the text below.

"Mikail Farrar, a Georgia FedEx carrier who asked the universe for help and Tony Hawk responded. It's sweeter than it sounds. Farrar was on his usual route this week in the Atlanta suburbs when a 6-year-old boy chased him down and asked him to send a skateboard to Tony Hawk. He didn't have the half-pipe hero's address, so Farrar took a chance and tried to reach Hawk through TikTok. It worked! The social media-savvy skateboarder coordinated with Farrar to send the boy's skateboard to Hawk's correct address and send the 6-year-old a new board. It's just further proof that people can be wonderful and that Tony Hawk is as kind as he is gnarly, dude."


In [ ]:
val text="""
Mikail Farrar, a Georgia FedEx carrier who asked the universe for help and Tony Hawk responded. It's sweeter than it sounds. Farrar was on his usual route this week in the Atlanta suburbs when a 6-year-old boy chased him down and asked him to send a skateboard to Tony Hawk. He didn't have the half-pipe hero's address, so Farrar took a chance and tried to reach Hawk through TikTok. It worked! The social media-savvy skateboarder coordinated with Farrar to send the boy's skateboard to Hawk's correct address and send the 6-year-old a new board. It's just further proof that people can be wonderful and that Tony Hawk is as kind as he is gnarly, dude.
"""
Sentiment.compute(text)

As we have seen in the Spire example, all jar files (one of the classes implemented in the jar is just a proxy for the whole jar) have to be declared as dependencies. In this particular case, we also have a file storing the models. That file, i.e. the *Models* file will be listed separately, since it doesn't implement any classes.

In [ ]:
addClass(session,"edu.stanford.nlp.pipeline.StanfordCoreNLP")
addClass(session,"org.ejml.simple.SimpleBase")
addClass(session,"org.ejml.data.Matrix")
addClass(session,"org.ejml.dense.row.CommonOps_DDRM")

session.addDependency("/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/edu/stanford/nlp/stanford-corenlp/4.2.2/stanford-corenlp-4.2.2-models.jar")

val sentimentUdf=udf((s:String) => Sentiment.compute(s))


In [ ]:
val sourceTextDf=session.createDataFrame(Seq(text)).toDF("TEXT")

sourceTextDf
    .withColumn("sentiment",callBuiltin("parse_json",sentimentUdf(col("TEXT"))))
    .select(col("sentiment"))
    .show();

# Conclusion

In this part of the series we have learned how to visualize datasets directly in a Jupyter notebook using the plotly-scala library. We also have learned how easy it is to enhance Snowflakes capabilities by creating Scala UDFs with third party libraries and arbitrary custom code. In the last installment we will combine all features we have seen in the previous two notebooks and will build a solution for an end-end ML based usecase.